# Test Metrics

## Import and Configuration

In [3]:
import psnr, rmse, vif, fsim, ssim, msssim, vsi
import numpy as np
import skimage.io as io
from skimage.color import rgb2gray
import torch
import torchvision

In [11]:
file_path_img_r = '../../TID2013/reference_images/I01.bmp'
file_path_img_m = '../../TID2013/distorted_images/I01_01_2.bmp'

img_r = io.imread(file_path_img_r)
img_m = io.imread(file_path_img_m)

img_r_gray = rgb2gray(img_r)
img_m_gray = rgb2gray(img_m)

img_r_tensor = torch.tensor(img_r).permute(2, 0, 1).unsqueeze(0)
img_m_tensor = torch.tensor(img_m).permute(2, 0, 1).unsqueeze(0)

img_r_tensor_gray = torchvision.transforms.Grayscale()(img_r_tensor)
img_m_tensor_gray = torchvision.transforms.Grayscale()(img_m_tensor)

In [6]:
img_r.shape

(384, 512, 3)

In [7]:
img_r_tensor.shape

torch.Size([1, 3, 384, 512])

## Calculations

### PSNR

In [19]:
metric = psnr.PSNR(data_range=255, normalize=False, batch=False)
metric.score(img_r, img_m)
metric.score_val

30.07379483937222

--> code OK, value OK

### RMSE

In [20]:
metric = rmse.RMSE(data_range=255, normalize=False)
metric.score(img_r, img_m)
metric.score_val

7.995588569810916

--> code OK, value probably OK

### VIFp

In [14]:
metric = vif.VIFp(data_range=255, normalize=False, chromatic=True)
metric.score(img_r, img_m)
metric.score_val

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

--> code OK, value **NOT** OK

In [20]:
# sewar VIFp
from sewar import vifp
print(vifp(img_r_gray, img_m_gray, sigma_nsq=3.2))

# piq VIFp
from piq import vif_p as piq_vif_p
print(piq_vif_p(img_r_tensor_gray, img_m_tensor_gray, data_range=255, sigma_n_sq=3.2))

# xdesign VIFp
from xdesign.metrics.fullref import vifp as xdesign_vifp
print(xdesign_vifp(img_m_gray, img_r_gray, L =255)[1].mean())

# # IQA-pytorch VIFp
# from IQA_pytorch import VIF
# from torch.nn.functional import normalize
# metric = VIF()
# score = metric(normalize(img_r_tensor.float()), normalize(img_m_tensor.float()), as_loss=False)

0.9983500921124459
tensor(0.6078)
0.9075681824869333


### FSIM

In [23]:
metric_fsim_m = fsim.FSIM(data_range=255, normalize=False, chromatic=True)
metric_fsim_m.score(img_r, img_m)
metric_fsim_m.score_val

0.9878281950950623

--> code OK, value OK

### SSIM

In [24]:
metric_ssim_m = ssim.SSIM(data_range=255, normalize=False)
metric_ssim_m.score(img_r, img_m, gaussian_weights=True, use_sample_covariance=False, sigma=1.5, channel_axis=2)
metric_ssim_m.score_val

0.8574712126318319

--> code OK, value **NOT** OK

In [13]:
# sewar ssim
from sewar import ssim
print('sewar: ', ssim(img_r, img_m))

# piq ssim
from piq import ssim as piq_ssim
print('piq: ', piq_ssim(img_r_tensor_gray, img_m_tensor_gray, data_range=255))

# skimage ssim
from skimage.metrics import structural_similarity as skimage_ssim
print('skimage: ', skimage_ssim(img_r, img_m, data_range=255, gaussian_weights=True, use_sample_covariance=False, sigma=1.5, channel_axis=2))

# pytorch-msssim ssim
from pytorch_msssim import ssim as pytorch_msssim_ssim
print('pytorch_msssim: ', pytorch_msssim_ssim(img_r_tensor_gray, img_m_tensor_gray, data_range=255))

# pytorch-ignite ssim
from ignite.metrics import SSIM
metric = SSIM(data_range=255)
metric.update((img_r_tensor_gray, img_m_tensor_gray))
print('pytorch-ignite: ', metric.compute())

# xdesign ssim
from xdesign.metrics import ssim as xdesign_ssim
print('xdesign: ', xdesign_ssim(img_r_gray, img_m_gray, L=255)[1])

# IQA-pytorch ssim
from IQA_pytorch import SSIM
from torch.nn.functional import normalize
metric = SSIM()
print('IQA-pytorch: ', metric(normalize(img_r_tensor_gray.float()), normalize(img_m_tensor_gray.float()), as_loss=False))

sewar:  (0.8909138757539132, 0.8909517569398758)
piq:  tensor(0.9807)
skimage:  0.8574712126318319
pytorch_msssim:  tensor(1.)
pytorch-ignite:  0.921117043856763
xdesign:  0.9999465399884802
IQA-pytorch:  tensor([0.9990])


### MSSSIM

In [26]:
metric_msssim_m = msssim.MSSSIM(data_range=255, normalize=False)
metric_msssim_m.score(img_r, img_m)
metric_msssim_m.score_val

(0.9821697985210762+0j)

--> code OK, value **NOT** OK

In [27]:
# sewar msssim
from sewar import msssim
print(msssim(img_r, img_m))

# piq msssim
from piq import multi_scale_ssim as piq_msssim
print(piq_msssim(img_r_tensor, img_m_tensor, data_range=255))

# pytorch-msssim msssim
try:
    from pytorch_msssim import ms_ssim as pytorch_msssim_msssim
    print(pytorch_msssim_msssim(img_r_tensor, img_m_tensor, data_range=255))
except:
    pass

# xdesign msssim
from xdesign.metrics import msssim as xdesign_msssim
print(xdesign_msssim(img_r_gray, img_m_gray, L=255)[1])

# IQA-pytorch msssim
from IQA_pytorch import MS_SSIM
from torch.nn.functional import normalize
metric = MS_SSIM()
print(metric(normalize(img_r_tensor.float()), normalize(img_m_tensor.float()), as_loss=False))

(0.9821697985210762+0j)
tensor(0.9775)
0.999975820750989
tensor([0.8833])


### VSI

In [28]:
metric = vsi.VSI(data_range=255, normalize=False)
metric.score(img_r, img_m)
metric.score_val

0.9948647022247314

--> code OK, value OK

## Conclusion

| Metric | Code OK | Value OK   |
| --- |---------|------------|
| PSNR | OK      | OK         |
| RMSE | OK      | OK         |
| VIFp | OK      | **NOT** OK |
| FSIM | OK      | OK         |
| SSIM | OK      | **NOT** OK |
| MSSSIM | OK      | **NOT** OK |
| VSI | OK      | OK         |
| MAD | --      | --         |
| GSM | --      | --         |
| SFF | --      | --         |
| CNR | --      | --         |
| Ma | --      | --         |
| PI | --      | --         |
| NIQE | --      | --         |